# Flatten tiles

# table of content
1) [Flattening a tile](#flattening-a-tile)
2) [Visualize the removed points](#visualize-the-removed-points)

### Dependencies and general utils

In [18]:
# pip install scipy

In [19]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import open3d as o3d
import laspy
# import pdal
import json
import scipy
import copy
import pickle
from tqdm import tqdm
from scipy.spatial import cKDTree

### Flattening a tile

#### example

In [20]:
def remove_duplicates(laz_file):
    # Find pairs of points
    coords = np.round(np.vstack((laz_file.x, laz_file.y, laz_file.z)),2).T
    tree_B = cKDTree(coords)
    pairs = tree_B.query_pairs(1e-2)

    # Create the mask with dupplicates
    mask = [True for i in range(len(coords))]
    for pair in pairs:
        mask[pair[1]] = False

    # Remove the dupplicates from the file
    laz_file.points = laz_file.points[mask]

In [21]:
tile_src = r"..\data\flattening_testing\color_grp_full_tile_128.laz"
tile_src = r"..\data\flattening_corrections\test\color_grp_full_tile_128.laz"
laz = laspy.read(tile_src)
print(len(laz))
remove_duplicates(laz)
print(len(laz))
laz.write(tile_src)
points = np.vstack((laz.x, laz.y, laz.z)).T
points_flatten = copy.deepcopy(points)
points_interpolated = copy.deepcopy(points)
print(list(laz.point_format.dimension_names))
print(points.shape)

388709
388709
['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'synthetic', 'key_point', 'withheld', 'overlap', 'scanner_channel', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'user_data', 'scan_angle', 'point_source_id', 'gps_time', 'red', 'green', 'blue']
(388709, 3)


In [22]:
grid_size=10
# Divide into tiles and find local minimums
#   _Create grid and find min Z in each cell
x_min, y_min = np.min(points[:, :2], axis=0)
x_max, y_max = np.max(points[:, :2], axis=0)

x_bins = np.append(np.arange(x_min, x_max, grid_size), x_max)
y_bins = np.append(np.arange(y_min, y_max, grid_size), y_max)

grid = {i:{j:[] for j in range(y_bins.size - 1)} for i in range(x_bins.size -1)}
for _, (px, py, pz) in tqdm(enumerate(points), total=len(points)):
    xbin = np.clip(0, (px - x_min) // grid_size, x_bins.size - 1)
    ybin = np.clip(0, (py - y_min) // grid_size, y_bins.size - 1)
    grid[xbin][ybin].append((px, py, pz))


100%|██████████| 388709/388709 [00:04<00:00, 88591.92it/s]


In [23]:
# Create grid_min
grid_used = np.zeros((x_bins.size - 1, y_bins.size - 1))
lst_grid_min = []
lst_grid_min_pos = []
for x in grid.keys():
    for y in grid[x].keys():
        if np.array(grid[x][y]).shape[0] > 0:
            grid_used[x, y] = 1
            # print(np.argmin(np.array(grid[x][y])[:,2]))
            lst_grid_min.append(np.min(np.array(grid[x][y])[:,2]))
            arg_min = np.argmin(np.array(grid[x][y])[:,2])
            lst_grid_min_pos.append(np.array(grid[x][y])[arg_min,0:2])
        else:
            grid_used[x, y] = 0
print(grid_used)
arr_grid_min_pos = np.vstack(lst_grid_min_pos)
print(arr_grid_min_pos.shape)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
(100, 2)


In [24]:
# Interpolate
points_xy = np.array(points)[:,0:2]
interpolated_min_z = scipy.interpolate.griddata(arr_grid_min_pos, np.array(lst_grid_min), points_xy, method="cubic", fill_value=-1)

mask_valid = np.array([x != -1 for x in list(interpolated_min_z)])
points_interpolated = points_interpolated[mask_valid]
points_interpolated[:, 2] = interpolated_min_z[mask_valid]

print(f"Original number of points: {points.shape[0]}")
print(f"Interpollated number of points: {points_interpolated.shape[0]} ({int(points_interpolated.shape[0] / points.shape[0]*100)}%)")


Original number of points: 388709
Interpollated number of points: 335358 (86%)


In [25]:
save = False

In [26]:
# save mask
with open(tile_src.split('.laz')[0] + f"_mask_{grid_size}m.pcl", '+wb') as file:
    pickle.dump(mask_valid, file)

In [27]:
# save floor
filtered_points = {dim: getattr(laz, dim)[mask_valid] for dim in laz.point_format.dimension_names}
header = laspy.LasHeader(point_format=laz.header.point_format, version=laz.header.version)
new_las = laspy.LasData(header)

#   _Assign filtered and modified data
for dim, values in filtered_points.items():
    setattr(new_las, dim, values)
# new_las.xyz = points_interpolated
setattr(new_las, 'x', points_interpolated[:,0])
setattr(new_las, 'y', points_interpolated[:,1])
setattr(new_las, 'z', points_interpolated[:,2])

print(len(new_las))
#   _Save new file
new_las.write(tile_src.split('.laz')[0] + f"_floor_{grid_size}m.laz")
print("Saved file: ", tile_src.split('.laz')[0] + f"_floor_{grid_size}m.laz")


# if save:
#     pcd.points = o3d.utility.Vector3dVector(points_interpolated)
#     o3d.io.write_point_cloud(tile_src.split('.pcd')[0] + f"_floor_{grid_size}m.pcd", pcd, write_ascii=True)

335358
Saved file:  ..\data\flattening_corrections\test\color_grp_full_tile_128_floor_10m.laz


In [28]:
test_laz = laspy.read(tile_src.split('.laz')[0] + f"_floor_{grid_size}m.laz")
print(len(test_laz))

335358


In [ ]:
# Flatten
points_flatten = points_flatten[mask_valid]
points_flatten[:,2] = points_flatten[:,2] - points_interpolated[:,2]
# points_flatten[:,2] = np.clip(0, points_flatten[:,2] - points_interpolated[:,2], np.inf)

filtered_points = {dim: getattr(laz, dim)[mask_valid] for dim in laz.point_format.dimension_names}
header = laspy.LasHeader(point_format=laz.header.point_format, version=laz.header.version)
new_las = laspy.LasData(header)

#   _Assign filtered and modified data
for dim, values in filtered_points.items():
    setattr(new_las, dim, values)
# new_las.xyz = points_flatten
setattr(new_las, 'x', points_flatten[:,0])
setattr(new_las, 'y', points_flatten[:,1])
setattr(new_las, 'z', points_flatten[:,2])

#   _Save new file
new_las.write(tile_src.split('.laz')[0] + f"_flatten_{grid_size}m.laz")
print("Saved file: ", tile_src.split('.laz')[0] + f"_flatten_{grid_size}m.laz")

# if save:
#     pcd.points = o3d.utility.Vector3dVector(points_flatten)
    
#     o3d.io.write_point_cloud(tile_src.split('.pcd')[0] + f"_flatten_{grid_size}m.pcd", pcd, write_ascii=True)


X
[257163674 257163634 257163595 ... 257173391 257173381 257173391]
Y
[109684818 109684764 109684714 ... 109692853 109692877 109692927]
Z
[187797 187817 187837 ... 185728 185689 185664]
intensity
[41891 43690 42919 ... 35466 35209 35980]
return_number
<SubFieldView([1 1 1 ... 1 1 1])>
number_of_returns
<SubFieldView([1 1 1 ... 1 1 1])>
synthetic
<SubFieldView([0 0 0 ... 0 0 0])>
key_point
<SubFieldView([0 0 0 ... 0 0 0])>
withheld
<SubFieldView([0 0 0 ... 0 0 0])>
overlap
<SubFieldView([0 0 0 ... 0 0 0])>
scanner_channel
<SubFieldView([0 0 0 ... 0 0 0])>
scan_direction_flag
<SubFieldView([0 0 0 ... 0 0 0])>
edge_of_flight_line
<SubFieldView([0 0 0 ... 0 0 0])>
classification
[ 3  3  3 ...  5 21  5]
user_data
[0 0 0 ... 0 0 0]
scan_angle
[0 0 0 ... 0 0 0]
point_source_id
[0 0 0 ... 0 0 0]
gps_time
[0. 0. 0. ... 0. 0. 0.]
red
[41634 43690 42919 ... 38807 37265 38807]
green
[41377 43433 42405 ... 34438 34438 34952]
blue
[45489 47288 46260 ... 32896 34438 34695]
Saved file:  ..\data\flatte

### Visualize the removed points

In [ ]:
# # Find removed points
# removed_points = copy.deepcopy(points)
# removed_points = removed_points[~mask_valid]

In [ ]:
# # Visualize removed points
# pcd_new = o3d.geometry.PointCloud()
# pcd_new.points = o3d.utility.Vector3dVector(removed_points)
# o3d.visualization.draw_geometries([pcd_new])